In [1]:
import multiprocessing
from tqdm.notebook import tqdm
import numpy as np
import scipy.stats as st
import numba

# Plotting modules
import bokeh.io
import bokeh.plotting
import biocircuits

bokeh.io.output_notebook()

# Line profiler (can install with conda install line_profiler)
%load_ext line_profiler

Loading BokehJS ...

SyntaxError: unexpected EOF while parsing (<ipython-input-1-ed3a9fbe3df0>, line 2)

I need to construct some sort of growth model.

- Cell volume: $C_V$
- Cell volume increase amount: $\delta$
- Nutrients: $n$
- Growth rate: $g$
$$\require{mhchem}$$
$$\ce{n ->[g] C_{V}}$$

In [40]:
#                           C_V_1, C_V_2, n_1, n_2
growth_update = np.array([[     1,     0,  -1,   0],
                          [     0,     1,   0,  -1]], # Growth
                          dtype = int)

def growthPropensity(propensities, variables, t, g):
    # Extract variables
    C_V_1, C_V_2, n_1, n_2 = variables
    
    # Update propensities
    propensities[0] = n_1 * g # C_V_1 Growth
    propensities[1] = n_2 * g # C_V_2 Growth


def gillespie_draw(propensity_func, propensities, variables, t, args=()):
    # 1) Calculate new propensites
    propensity_func(propensities, variables, t, *args)
    
    # 2) Sum propensities
    propensity_sum = propensities.sum()
    
    # 3) Compute next time
    dt = np.random.exponential(1.0 / propensity_sum)
    
    # 4) Compute reaction probabilites from propensities
    reaction_probabilites = propensities / propensity_sum
    
    # 5) Draw a random reaction
    chosen_reaction = sample_discrete(reaction_probabilites)
    
    return chosen_reaction, dt

# Now we will construct a functions to run the gilespie algorithim
def gillespie_algorithim(propensity_function, update_array, initial_conditions, time_points, args=()):
    # Initialise output
    pop_out = np.empty((len(time_points), update_array.shape[1]), dtype = int)

    # Initialise and perform simulation
    index = 0
    time_index = 1
    time = time_points[0]
    variables = initial_conditions.copy()
    pop_out[0,:] = variables
    propensities = np.zeros(update_array.shape[0])
    while index < len(time_points):
        while time < time_points[time_index]:
            # Draw reaction and time step
            reaction, dt = gillespie_draw(propensity_function, propensities, variables, time, args)

            # Update variables
            variables_previous = variables.copy()
            variables += update_array[reaction, :]
#             print(variables[0])
            if variables[0] >= 10:
                variables[0], variables[1] = variables[0]/2, variables[0]/2
                variables[2], variables[3] = variables[2]/2, variables[2]/2
            # Increment time
            time += dt

        # Update index
        index = np.searchsorted(time_points > time, True)

        # Update the variables
        pop_out[time_index:min(index, len(time_points))] = variables_previous

        # Increment index
        time_index = index

    return pop_out

In [43]:
# Now we can run the simulation
#       g = 1
args = (0.1,)

time_points = np.linspace(0, 50, 101)
#                              C_V, n
initial_conditions = np.array([  5, 0, 100, 0],  dtype = int)
iterations = 100

# See random number for reproducibility
np.random.seed(420)

# Initialise output array
out_variables = np.empty((iterations, len(time_points), 4), dtype = int)

# Run the calculations
for index in tqdm(range(iterations)):
    out_variables[index, :, :] = gillespie_algorithim(growthPropensity, growth_update, initial_conditions, time_points, args = args)

  0%|          | 0/100 [00:00<?, ?it/s]

<ipython-input-40-d5e94a9c8725>:34: RuntimeWarning: divide by zero encountered in double_scalars
  dt = np.random.exponential(1.0 / propensity_sum)
<ipython-input-40-d5e94a9c8725>:37: RuntimeWarning: invalid value encountered in true_divide
  reaction_probabilites = propensities / propensity_sum


In [45]:
# Now we can plot the results
plots = [bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'Cell Volume 1'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'Cell Volume 2'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'Nutrients 1'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'Nutrients 2')]
         
for index in range(4):
    for out_variables_current in out_variables[:,:,index]:
        plots[index].line(time_points, out_variables_current, line_width = 0.3, alpha = 0.2, line_join = 'bevel')
    plots[index].line(time_points, out_variables[:,:,index].mean(axis = 0), line_width = 6, color = 'orange', line_join = 'bevel')
#Link axes
plots[0].x_range = plots[1].x_range

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols = 2))

In [14]:
delta, g

tuple

In [13]:
type(tuple(1))

TypeError: 'int' object is not iterable

In [894]:
CS_obj.all_cells[0]

In [2]:
class Cell_Stack:
    def __init__(self):
        self.all_cells = (cell_single(),)
        self.grand_reaction_array = np.array([])
        self.update_reaction_array()
        self.ID_list = []
        self.set_ID(self.all_cells[0] ,'0')

    def set_ID(self, curr_cell, new_ID):
        curr_cell.ID += str(new_ID)
        
    def get_variables(self):
        all_variables = []
        for curr_cell in self.all_cells:
            all_variables.extend(curr_cell.get_variables())
        return all_variables
    
    def get_variables_labels(self):
        return self.all_cells.get_variables_labels()
    
    def update_variables(self, update):
        len_vars = len(self.all_cells[0].variables.values())
        for ind in range(0, len(update), len_vars):
            cell_index = int(np.floor(ind/len_vars))
            self.all_cells[cell_index].update_variables(update[ind:ind+len_vars])
            
            
        
    def check_split(self):
        ind = 0
        did_split_occur = False
        while ind < len(self.all_cells):
            curr_cell = self.all_cells[ind]
            if curr_cell.variables["V"] > 100:
                out = curr_cell.split_cell()
                self.all_cells[ind].set_variables(out[0,:])
                self.all_cells = self.all_cells[:ind] + (cell_single(),) + self.all_cells[ind:]
                self.all_cells[ind].set_variables(out[1,:])
                self.set_ID(self.all_cells[ind], self.all_cells[ind+1].ID + '0')
                self.set_ID(self.all_cells[ind+1], '1')
                ind = -1
                did_split_occur = True
            ind += 1
        return did_split_occur
                
    def get_stack_propensities(self):
        propensities = np.array([])
        for curr_cell in self.all_cells:
            propensities = np.concatenate((propensities, curr_cell.get_propensities()))
        return propensities
        
    def update_reaction_array(self):
        self.grand_reaction_array = self.all_cells[0].reaction_array
        [r_y, r_x] = np.shape(self.grand_reaction_array)
        cra_y, cra_x = 0, 0
        for curr_cell in self.all_cells[1:]:
            cra_y += r_y
            cra_x += r_x
            self.grand_reaction_array = np.concatenate((np.concatenate((self.grand_reaction_array, np.zeros([cra_y, r_x])), axis=1), np.concatenate((np.zeros([r_y, cra_x]), curr_cell.reaction_array), axis=1)))
    
class cell_single:
    def __init__(self):
        self.variables = {
            "V" : 50,
            "n_i" : 500,
        }
        self.V = [50, 51]
        self.rates = {
            "g" : 0.1
        }
        
        self.ID = ''
        
        #                               V, n_i
        self.reaction_array = np.array([[1, -1], # R1: Consuption of n_i to make V
                                       ], dtype = int)
        
    def get_propensities(self):
#         print(self.variables["n_i"])
#         print(self.rates["g"])
        return np.array([self.variables["n_i"] * self.rates["g"], # R1: Consuption of n_i to make V
                                     ])
    def get_variables(self):
        return list(self.variables.values())
    
    def get_variables_labels(self):
        return list(self.variables.keys())
    
    def set_variables(self, new_variables):
        for ind, curr_key in enumerate(self.variables.keys()):
            self.variables.update({curr_key : new_variables[ind]})
            
    def update_variables(self, update_variables):
        for ind, curr_key in enumerate(self.variables.keys()):
            self.variables.update({curr_key : self.variables[curr_key] + update_variables[ind]})
    
    def split_cell(self):
        out = np.zeros([2, len(self.variables)])
        for ind, key in enumerate(self.variables.keys()):
            r = np.random.uniform(0.25, 0.75)
            out[0, ind] = int(round(r*self.variables[key]))
            out[1, ind] = int(round((1-r)*self.variables[key]))
        return out
    
        

In [5]:
def sample_discrete(probs):
    """Randomly sample an index with probability given by probs."""
    # Generate random number
    q = np.random.rand()
    
    # Find index
    i = 0
    p_sum = 0.0
    while p_sum < q:
        p_sum += probs[i]
        i += 1
    return i - 1

def gillespie_draw(propensities):
#     print("propensities=",propensities)
    # 2) Sum propensities
    propensity_sum = propensities.sum()
    
    # 3) Compute next time
    dt = np.random.exponential(1.0 / propensity_sum)
    
    # 4) Compute reaction probabilites from propensities
    reaction_probabilites = propensities / propensity_sum
    
    # 5) Draw a random reaction
    chosen_reaction = sample_discrete(reaction_probabilites)
    
    return chosen_reaction, dt

In [6]:
CS_obj = Cell_Stack()

time_points = np.linspace(0, 20, 20)
pop_out = np.empty((len(time_points), CS_obj.grand_reaction_array.shape[1]), dtype = int)

# Initialise and perform simulation
index = 0
time_index = 1
time = time_points[0]
variables = CS_obj.get_variables()
pop_out[0,:] = variables
propensities = np.zeros(CS_obj.grand_reaction_array.shape[1])
while index < len(time_points):
    while time < time_points[time_index]:
        # Draw reaction and time step
#         print(CS_obj.get_stack_propensities())
        reaction, dt = gillespie_draw(CS_obj.get_stack_propensities())
#         print(reaction)
        # Update variables
        variables_previous = CS_obj.get_variables()
#         print(CS_obj.grand_reaction_array[reaction, :])
        CS_obj.update_variables(CS_obj.grand_reaction_array[reaction, :])
        
        
        
        # Increment time
        time += dt
        

    # Update index
    index = np.searchsorted(time_points > time, True)

    # Update the variables
    pop_out[time_index:min(index, len(time_points))] = variables_previous

    # Increment index
    time_index = index
    
    CS_obj.check_split()
    CS_obj.update_reaction_array()
    pop_out = np.concatenate((pop_out, np.zeros([np.shape(pop_out)[0], np.shape(CS_obj.get_variables())[0]-np.shape(pop_out)[1]])), axis = 1)
    

In [7]:
# Now we can plot the results
plots = [bokeh.plotting.figure(plot_width = 500, plot_height = 300, x_axis_label = 'time', y_axis_label = 'Cell Volume 1'),
         bokeh.plotting.figure(plot_width = 500, plot_height = 300, x_axis_label = 'time', y_axis_label = 'Nutrients 1')]
         
    
colours = bokeh.palettes.viridis(pop_out.shape[1])
for ind in range(0, pop_out.shape[1], 2):
    plots[0].line(time_points, pop_out[:,ind], line_width = 1, alpha = 1, line_join = 'bevel', color = colours[ind])

for ind in range(1, pop_out.shape[1]-1, 2):
    plots[1].line(time_points, pop_out[:,ind], line_width = 1, alpha = 1, line_join = 'bevel', color = colours[ind-1])
#Link axes
plots[0].x_range = plots[1].x_range

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols = 2))

In [977]:
[curr_cell.ID for curr_cell in CS_obj.all_cells]

['0000', '0001', '0010', '0011', '010', '0110', '0111']

In [928]:
'0' -> '00' + '01'

['0', '0']

In [978]:
pop_out

array([[ 50., 500.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.],
       [109., 441.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.],
       [ 94., 180.,  56., 220.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.],
       [111., 163.,  82., 194.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.],
       [ 80.,  62.,  43.,  89., 107., 169.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.],
       [ 88.,  54.,  56.,  76.,  59.,  51.,  64., 102.,   0.,   0.,   0.,
          0.,   0.,   0.],
       [ 94.,  48.,  61.,  71.,  63.,  47.,  75.,  91.,   0.,   0.,   0.,
          0.,   0.,   0.],
       [ 98.,  44.,  72.,  60.,  66.,  44.,  84.,  82.,   0.,   0.,   0.,
          0.,   0.,   0.],
       [104.,  38.,  78.,  54.,  71.,  39.,  90.,  76.,   0.,   0.,   0.,
          0.,   0.,   0.],
       [ 45.,  12.,  64.,  21.,  84.,  48.,  75.,  35.,  95.,  71.,   0.,
          0.,   0

In [795]:
a = np.ones([1,3])
print(a)

[[1. 1. 1.]]


In [960]:
ind = 1
a = (1,2)
a[:ind] + (9,) + a[ind:]

(1, 9, 2)

In [801]:
pop_out

array([[ 50, 500],
       [ 99, 451],
       [144, 406],
       [185, 365],
       [225, 325],
       [253, 297],
       [291, 259],
       [316, 234],
       [342, 208],
       [360, 190]])

In [806]:
CS_obj.check_split()
np.shape(CS_obj.get_variables())

(10,)

In [816]:
np.concatenate((pop_out, np.zeros([np.shape(pop_out)[0], np.shape(CS_obj.get_variables())[0]-np.shape(pop_out)[1]])), axis = 1)

array([[ 50., 500.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [ 99., 451.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [144., 406.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [185., 365.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [225., 325.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [253., 297.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [291., 259.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [316., 234.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [342., 208.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [360., 190.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

In [810]:
np.shape(pop_out)[0]

10

In [812]:
np.shape(CS_obj.get_variables())[0]

10

In [881]:
a = str(1)

In [882]:
print(a)

1


In [885]:
a + str(2)

'12'